In [ ]:
import os, sys
import numpy as np
import dask
import xarray as xr
from matplotlib import pyplot as plt
%matplotlib inline

from mitequinox.utils import *

In [ ]:
from dask_jobqueue import PBSCluster
local_dir = os.getenv('TMPDIR')
cluster = PBSCluster(local_directory=local_dir)
#print(cluster.job_script())
w = cluster.start_workers(20)

In [ ]:
# get dask handles and check dask server status
from dask.distributed import Client
client = Client(cluster)

In [13]:
client

Client Scheduler: tcp://10.148.1.13:54710 Dashboard: http://10.148.1.13:8787/status,Cluster Workers: 24 Cores: 96 Memory: 1200.00 GB


--- 

# automatic rewriting of all variables (for spectral calculations)

# transposed data: (i,j,time), 1 file per face, time windows

In [14]:
face_60 = [1, 2, 4, 5, 7, 8, 10, 11]
face_60 = [5]

print(face_60)

[5]


In [15]:
V = ['SSU']

transpose = True # False untested

if transpose:
    Nt = 24*10*3 # time windows to consider
    out_dir = scratch+'/mit_nc_t/'
    #out_dir = 'home1/datawork/xyu/'
    fsize_bound = 15*1e9*3
else:
    Nt = 1
    out_dir = scratch+'/mit_nc/'    
    fsize_bound = 60*1e6    

for v in V:
    #
    data_dir = root_data_dir+v+'/'
    iters, time = get_iters_time(v, data_dir, delta_t=25.)
    #
    #it = np.arange(time.size/Nt-1).astype(int)*Nt
    it = np.arange(1).astype(int)*Nt # tmp
    assert it[-1]+Nt<time.size
    #
    p = 'C'
    vdrop = ['XC','YC','Depth','rA']
    if v is 'SSU':
        p = 'W'
        vdrop = ['dxC','dyG','Depth','rAw']        
    elif v is 'SSV':
        p = 'S'
        vdrop = ['dxG','dyC','rAs']       
    #
    ds = get_compressed_data(v, data_dir, grid_dir, iters=iters, time=time, client=client, point=p)
    #ds = ds.chunk({'face': 1})
    #
    #for face in range(ds['face'].size):
    for face in face_60:   
        for i, t in enumerate(it):
            #
            file_out = out_dir+'/%s_f%02d_t%02d.nc'%(v,face,i)
            if not os.path.isfile(file_out) or os.path.getsize(file_out) < fsize_bound:            
                dv = ds[v].isel(time=slice(t,t+Nt), face=face) 
                # should store grid data independantly in a single file
                #dv = dv.drop(['XC','YC','Depth','rA']).to_dataset()
                dv = dv.drop(['dyG','dxC','rAw']).to_dataset()

                #
                if transpose:
                    dv = dv.chunk({'time': dv['time'].size, 'i_g': 432, 'j': 432})
                    dv = dv.transpose('i_g','j','time')
                    chunksizes = [432, 432, dv['time'].size]
                else:
                    dv = dv.chunk({'i': 432, 'j': 432})
                    chunksizes = [1, 432, 432]
                #print(dv)
                #
                while True:
                    try:
                        %time dv.to_netcdf(file_out, mode='w', unlimited_dims=['time'], \
                                           encoding={'SSU': {'chunksizes': chunksizes}})
                    except:
                        print('Failure')
                    if os.path.isfile(file_out) and os.path.getsize(file_out)>fsize_bound:
                        #
                        print('face=%d / i=%d'%(face,i))
                        break
            else:
                print('face=%d / i=%d - allready processed'%(face,i))

/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:735: UserWarning: Couldn't find available_diagnostics.log in . Using default version.
  "in %s. Using default version." % data_dir)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/utils.py:336: UserWarning: Not sure what to do with rlev = L
  warnings.warn("Not sure what to do with rlev = " + rlev)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:249: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  for vname in ds:
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/dask/utils.py:1010: UserWarning: Deprecated, see dask.base.get_scheduler instead
  warnings.warn("Deprecated, see dask.base.get_scheduler inst

CPU times: user 1min 38s, sys: 5.77 s, total: 1min 44s
Wall time: 5min 41s
face=5 / i=0


--- 

# standard data layout: (face, j, i), 1 file per time (for movies)

In [ ]:
V = ['Eta', 'SST', 'SSS', 'SSU', 'SSV']
V = ['SSU']

Nt = 1
out_dir = '/home1/work/xyu/LLC4320/'  
print(out_dir)

fsize_bound = 13*60*1e6

for v in V:
    #
    data_dir = root_data_dir+v+'/'
    iters, time = get_iters_time(v, data_dir, delta_t=25.)
    #
    it = np.arange(time.size/Nt-1).astype(int)*Nt
    it = np.arange(3).astype(int)*Nt # tmp
    assert it[-1]+Nt<time.size
    #
    p = 'C'
    vdrop = ['XC','YC','Depth','rA']
    if v is 'SSU':
        p = 'W'
        vdrop = ['dxC','dyG','rAw']        
    elif v is 'SSV':
        p = 'S'
        vdrop = ['dxG','dyC','rAs']        
    #
    ds = get_compressed_data(v, data_dir, grid_dir, iters=iters, time=time, client=client, point=p)
    
    #ds = ds.chunk({'face': 1})
    #
    for i, t in enumerate(it):
        #
        file_out = out_dir+'/%s_t%04d.nc'%(v,i)
        if not os.path.isfile(file_out) or os.path.getsize(file_out) < fsize_bound:            
            dv = ds[v].isel(time=slice(t,t+Nt)) 
            # should store grid data independantly in a single file
            dv = dv.drop(vdrop).to_dataset()
            #
            #print(dv)
            #
            while True:
                try:
                    #print(dv)
                    %time dv.to_netcdf(file_out, mode='w')                    
                except:
                    print('Failure')
                if os.path.isfile(file_out) and os.path.getsize(file_out) > fsize_bound:
                    #
                    print('i=%d, iter=%d'%(i, iters[i].values))
                    break
        else:
            print('i=%d, iter=%d - allready processed'%(i, iters[i].values))

/home1/scratch/xyu/mit_nc_t/


/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:735: UserWarning: Couldn't find available_diagnostics.log in . Using default version.
  "in %s. Using default version." % data_dir)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/utils.py:336: UserWarning: Not sure what to do with rlev = L
  warnings.warn("Not sure what to do with rlev = " + rlev)
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/xmitgcm/mds_store.py:249: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  for vname in ds:
/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/dask/utils.py:1010: UserWarning: Deprecated, see dask.base.get_scheduler instead
  warnings.warn("Deprecated, see dask.base.get_scheduler inst

CPU times: user 1.22 s, sys: 80 ms, total: 1.3 s
Wall time: 10.4 s
i=0, iter=228096


/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/dask/utils.py:1010: UserWarning: Deprecated, see dask.base.get_scheduler instead
  warnings.warn("Deprecated, see dask.base.get_scheduler instead")


CPU times: user 724 ms, sys: 64 ms, total: 788 ms
Wall time: 4.65 s
i=1, iter=228240


/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/dask/utils.py:1010: UserWarning: Deprecated, see dask.base.get_scheduler instead
  warnings.warn("Deprecated, see dask.base.get_scheduler instead")


CPU times: user 732 ms, sys: 40 ms, total: 772 ms
Wall time: 4.75 s
i=2, iter=228384


/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/dask/utils.py:1010: UserWarning: Deprecated, see dask.base.get_scheduler instead
  warnings.warn("Deprecated, see dask.base.get_scheduler instead")


CPU times: user 732 ms, sys: 44 ms, total: 776 ms
Wall time: 4.73 s
i=3, iter=228528


/home1/datahome/xyu/.miniconda3/envs/equinox/lib/python3.6/site-packages/dask/utils.py:1010: UserWarning: Deprecated, see dask.base.get_scheduler instead
  warnings.warn("Deprecated, see dask.base.get_scheduler instead")


In [17]:
cluster.scheduler.close()

<Future finished result=None>